<a href="https://colab.research.google.com/github/rubanzasilva/tabular-playground-dataset/blob/main/tabularPlayground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install fastai
!pip3 install fastai

In [ ]:
!git clone https://github.com/rubanzasilva/tabular-playground-dataset.git


Cloning into 'tabular-playground-dataset'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 6 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
from fastai import *
from fastai.tabular import *
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import torch
from torch import nn, optim

In [9]:
!ls tabular-playground-dataset/

README.md  tabular-playground-series-mar-2022.zip


In [8]:
!unzip tabular-playground-dataset/tabular-playground-series-mar-2022.zip

Archive:  tabular-playground-dataset/tabular-playground-series-mar-2022.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [10]:
!ls

sample_data	       tabular-playground-dataset  train.csv
sample_submission.csv  test.csv


In [11]:
path = ('/')

In [14]:
train_df = pd.read_csv('train.csv')
train_df.head()

,row_id,time,x,y,direction,congestion
0,0,1991-04-01 00:00:00,0,0,EB,70
1,1,1991-04-01 00:00:00,0,0,NB,49
2,2,1991-04-01 00:00:00,0,0,SB,24
3,3,1991-04-01 00:00:00,0,1,EB,18
4,4,1991-04-01 00:00:00,0,1,NB,60


In [16]:
size1 = train_df.size
size1

5093010

In [17]:
shape = train_df.shape
shape

(848835, 6)